Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Firtyl, starting with the logistic model:

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
    
  beta_reg=tf.placeholder(tf.float32)
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases


  #compute loss and l2 loss
  l2_loss =beta_reg * (tf.nn.l2_loss(weights))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+l2_loss)
  
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_reg : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.769207
Minibatch accuracy: 7.0%
Validation accuracy: 14.4%
Minibatch loss at step 500: 2.785585
Minibatch accuracy: 76.6%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.869707
Minibatch accuracy: 77.3%
Validation accuracy: 78.1%
Minibatch loss at step 1500: 0.949160
Minibatch accuracy: 84.4%
Validation accuracy: 79.3%
Minibatch loss at step 2000: 0.810693
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 2500: 0.845272
Minibatch accuracy: 79.7%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.748594
Minibatch accuracy: 85.2%
Validation accuracy: 81.9%
Test accuracy: 89.0%


Secondly, the neural network will be regularized:

In [7]:
batch_size=128
num_hidden_nodes=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    #Input layer
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    #hidden layer
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
  #regularization parameter
  beta_reg=tf.placeholder(tf.float32)

  
  # Training computation.
  layer1=tf.matmul(tf_train_dataset, weights1)+biases1
  layer1 = tf.nn.relu(layer1)
  logits = tf.matmul(layer1, weights2) + biases2

    # compute losses
  l2_loss =beta_reg * (tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+l2_loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_layer1=tf.matmul(tf_valid_dataset, weights1)+biases1
  valid_layer1=tf.nn.relu(valid_layer1)    
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_layer1, weights2) + biases2)

  test_layer1=tf.matmul(tf_test_dataset, weights1)+biases1
  test_layer1=tf.nn.relu(test_layer1)    
  test_prediction = tf.nn.softmax(
    tf.matmul(test_layer1, weights2) + biases2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_reg : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 623.124329
Minibatch accuracy: 4.7%
Validation accuracy: 26.6%
Minibatch loss at step 500: 200.203979
Minibatch accuracy: 80.5%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 115.699181
Minibatch accuracy: 78.9%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 68.326195
Minibatch accuracy: 90.6%
Validation accuracy: 83.1%
Minibatch loss at step 2000: 41.231354
Minibatch accuracy: 90.6%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 25.125336
Minibatch accuracy: 90.6%
Validation accuracy: 85.4%
Minibatch loss at step 3000: 15.499150
Minibatch accuracy: 85.9%
Validation accuracy: 86.7%
Test accuracy: 93.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [13]:
batch_size=128
num_hidden_nodes=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    #Input layer
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    #hidden layer
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
  #regularization parameter
  beta_reg=tf.placeholder(tf.float32)

  
  # Training computation.
  layer1=tf.matmul(tf_train_dataset, weights1)+biases1
  layer1 = tf.nn.relu(layer1)
  logits = tf.matmul(layer1, weights2) + biases2

    # compute losses
  l2_loss =beta_reg * (tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+l2_loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_layer1=tf.matmul(tf_valid_dataset, weights1)+biases1
  valid_layer1=tf.nn.relu(valid_layer1)    
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_layer1, weights2) + biases2)

  test_layer1=tf.matmul(tf_test_dataset, weights1)+biases1
  test_layer1=tf.nn.relu(test_layer1)    
  test_prediction = tf.nn.softmax(
    tf.matmul(test_layer1, weights2) + biases2)

In [14]:
num_steps = 101
num_batches = 3

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = ((step % num_batches) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_reg : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 674.672363
Minibatch accuracy: 10.2%
Validation accuracy: 26.5%
Minibatch loss at step 2: 1885.928833
Minibatch accuracy: 29.7%
Validation accuracy: 33.4%
Minibatch loss at step 4: 1174.608032
Minibatch accuracy: 46.1%
Validation accuracy: 49.4%
Minibatch loss at step 6: 536.268433
Minibatch accuracy: 65.6%
Validation accuracy: 67.6%
Minibatch loss at step 8: 330.998138
Minibatch accuracy: 88.3%
Validation accuracy: 73.1%
Minibatch loss at step 10: 329.246277
Minibatch accuracy: 93.0%
Validation accuracy: 70.1%
Minibatch loss at step 12: 419.728210
Minibatch accuracy: 84.4%
Validation accuracy: 73.2%
Minibatch loss at step 14: 310.458923
Minibatch accuracy: 98.4%
Validation accuracy: 74.1%
Minibatch loss at step 16: 312.407440
Minibatch accuracy: 96.9%
Validation accuracy: 74.3%
Minibatch loss at step 18: 318.567078
Minibatch accuracy: 95.3%
Validation accuracy: 73.8%
Minibat

Strong overfitting happens, since there are so little examples that the model can learn to predict them perfectly.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
batch_size=128
num_hidden_nodes=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    #Input layer
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    #hidden layer
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  layer1=tf.matmul(tf_train_dataset, weights1)+biases1
  layer1 = tf.nn.relu(layer1)
  dropout_layer=tf.nn.dropout(layer1, 0.5)
  logits = tf.matmul(dropout_layer, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_layer1=tf.matmul(tf_valid_dataset, weights1)+biases1
  valid_layer1=tf.nn.relu(valid_layer1)    
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_layer1, weights2) + biases2)

  test_layer1=tf.matmul(tf_test_dataset, weights1)+biases1
  test_layer1=tf.nn.relu(test_layer1)    
  test_prediction = tf.nn.softmax(
    tf.matmul(test_layer1, weights2) + biases2)

In [12]:
num_steps = 101
num_batches = 3

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = step % num_batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 550.988892
Minibatch accuracy: 7.8%
Validation accuracy: 24.8%
Minibatch loss at step 2: 989.622864
Minibatch accuracy: 46.1%
Validation accuracy: 34.8%
Minibatch loss at step 4: 403.714539
Minibatch accuracy: 68.8%
Validation accuracy: 59.2%
Minibatch loss at step 6: 19.202570
Minibatch accuracy: 91.4%
Validation accuracy: 64.0%
Minibatch loss at step 8: 20.364906
Minibatch accuracy: 92.2%
Validation accuracy: 61.6%
Minibatch loss at step 10: 29.781658
Minibatch accuracy: 93.8%
Validation accuracy: 66.7%
Minibatch loss at step 12: 8.348515
Minibatch accuracy: 98.4%
Validation accuracy: 66.7%
Minibatch loss at step 14: 3.938969
Minibatch accuracy: 96.1%
Validation accuracy: 67.6%
Minibatch loss at step 16: 1.168028
Minibatch accuracy: 97.7%
Validation accuracy: 65.4%
Minibatch loss at step 18: 0.233244
Minibatch accuracy: 99.2%
Validation accuracy: 66.4%
Minibatch loss at ste

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


First step: Use both regularization and droput.

In [15]:
batch_size=128
num_hidden_nodes=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    #Input layer
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    #hidden layer
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
  #regularization parameter
  beta_reg=tf.placeholder(tf.float32)

  
  # Training computation.
  layer1=tf.matmul(tf_train_dataset, weights1)+biases1
  layer1 = tf.nn.relu(layer1)
  dropout_layer=tf.nn.dropout(layer1, 0.5)
  logits = tf.matmul(dropout_layer, weights2) + biases2

    # compute losses
  l2_loss =beta_reg * (tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+l2_loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_layer1=tf.matmul(tf_valid_dataset, weights1)+biases1
  valid_layer1=tf.nn.relu(valid_layer1)    
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_layer1, weights2) + biases2)

  test_layer1=tf.matmul(tf_test_dataset, weights1)+biases1
  test_layer1=tf.nn.relu(test_layer1)    
  test_prediction = tf.nn.softmax(
    tf.matmul(test_layer1, weights2) + biases2)

In [17]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_reg : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 849.559753
Minibatch accuracy: 7.8%
Validation accuracy: 30.1%
Minibatch loss at step 500: 213.008789
Minibatch accuracy: 76.6%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 119.371735
Minibatch accuracy: 72.7%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 69.281097
Minibatch accuracy: 78.9%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 41.445061
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 25.199787
Minibatch accuracy: 85.9%
Validation accuracy: 84.3%
Minibatch loss at step 3000: 15.539665
Minibatch accuracy: 82.8%
Validation accuracy: 85.0%
Minibatch loss at step 3500: 9.667274
Minibatch accuracy: 82.0%
Validation accuracy: 85.4%
Minibatch loss at step 4000: 6.009874
Minibatch accuracy: 86.7%
Validation accuracy: 85.8%
Minibatch loss at step 4500: 3.874226
Minibatch accuracy: 85.2%
Validation accuracy: 85.6%

Next step: introduce learning rate decay.

In [19]:
batch_size=128
num_hidden_nodes=1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    #Input layer
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    #hidden layer
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
  #regularization parameter
  beta_reg=tf.placeholder(tf.float32)

  
  # Training computation.
  layer1=tf.matmul(tf_train_dataset, weights1)+biases1
  layer1 = tf.nn.relu(layer1)
  dropout_layer=tf.nn.dropout(layer1, 0.5)
  logits = tf.matmul(dropout_layer, weights2) + biases2

    # compute losses
  l2_loss =beta_reg * (tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+l2_loss)
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.8)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
  valid_layer1=tf.matmul(tf_valid_dataset, weights1)+biases1
  valid_layer1=tf.nn.relu(valid_layer1)    
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_layer1, weights2) + biases2)

  test_layer1=tf.matmul(tf_test_dataset, weights1)+biases1
  test_layer1=tf.nn.relu(test_layer1)    
  test_prediction = tf.nn.softmax(
    tf.matmul(test_layer1, weights2) + biases2)

In [20]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta_reg : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 809.888794
Minibatch accuracy: 10.2%
Validation accuracy: 26.1%
Minibatch loss at step 500: 212.505783
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 1000: 121.319809
Minibatch accuracy: 74.2%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 72.319031
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 45.553677
Minibatch accuracy: 78.1%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 28.809502
Minibatch accuracy: 82.8%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 18.716404
Minibatch accuracy: 83.6%
Validation accuracy: 85.0%
Minibatch loss at step 3500: 12.360815
Minibatch accuracy: 82.0%
Validation accuracy: 85.6%
Minibatch loss at step 4000: 8.216643
Minibatch accuracy: 82.8%
Validation accuracy: 85.7%
Minibatch loss at step 4500: 5.664885
Minibatch accuracy: 84.4%
Validation accuracy: 86.